# What we mapped

With this notebook, and with [QGIS TimeManager plug-in](https://plugins.qgis.org/plugins/timemanager/), you will be able to create animated gif and movie to illustrate the work accomplished during a Mapathon event.

# Configuration

## Import libraries

In [77]:
## Import libraries needed for setting parameters of operating system 
import os
import sys

## Import library for temporary files creation 
import tempfile 

## Import Numpy library
import numpy as np

## Import Psycopg2 library (interection with postgres database)
import psycopg2 as pg

## Import Pandas library (View and manipulaiton of tables)
import pandas as pd

## User inputs

In [78]:
## Define a empty dictionnary for saving user inputs
user={}

In [95]:
## Enter the name of the new postgresqgl database
user["dbname"] = "Mapathon"
## Enter the postgresqgl username
user["user"] = "tais"
## Enter postgresqgl Password
user["password"] = "tais"
## Enter postgresqgl host
user["host"] = "localhost"
## Enter DB port
user["port"] = "5432"
## Enter postgresqgl schema
user["schema"] = "belgianmapath2018"

Here after:
- Enter the path to the main directory of Postgresql.

In [80]:
## Enter the path to the osm2pgsql folder
user["pgsqlfolder"]="/Library/PostgreSQL/9.6/bin"
## Add environment variables for Postgresql
os.environ['PATH'] = user["pgsqlfolder"] + os.pathsep + os.environ['PATH']

Here after:
- Enter the path to the directory where osm2pgsql in located. 
The command-line based program "osm2pgsql" have to be installed on your computer. See [the official wiki page](http://wiki.openstreetmap.org/wiki/Osm2pgsql) for more information. For Windows user, you have to [dowload the osm2pgsql executable](https://ci.appveyor.com/project/openstreetmap/osm2pgsql/history). Go to the page and find the latest 'artifact'.  




In [81]:
## Enter the path to the osm2pgsql folder
user["osm2pgsqlfolder"]="/usr/local/Cellar/osm2pgsql/0.94.0_3"
## Add environment variables for osm2pgsql
os.environ['PATH'] = user["osm2pgsqlfolder"] + os.pathsep + os.environ['PATH']

In [82]:
# Define the path to the osm2pgsql default.style file
user["stylefile"]="/Users/taisgrippa/Dropbox/BROL/Mapathon_Tais/2018_gif/default_OSMmetadata.style"

Here after:
- Enter the path to the folder where the .osm file covering your area of interest will be downloaded. 
OSM data downloading is automated in this script. In case you would manage yourself the retrieving of OSM data, please read the [official wiki page](http://wiki.openstreetmap.org/wiki/Downloading_data) for that purpose. In that case some parts of this notebook should be adapted.

In [83]:
## Enter the path to the folder where the .osm file will be downloaded
user["osmfolder"]="/Users/taisgrippa/Dropbox/BROL/Mapathon_Tais/2018_gif/Mapathon2018_belgiumuni"
## Enter the prefix you want to be used in the PostGIS DB
user["prefixosm"]="osm"

Here after:
- Enter the path to a .shp file (a single polygon feature) corresponding to the area of interest (AOI) which has been mapped. Please, be sure the projection is EPSG 3857.

In [84]:
## Enter the path to the .shp file corresponding to the area of interested which has been mapped
user["aoi_geojson"]="/Users/taisgrippa/Dropbox/BROL/Mapathon_Tais/2018_gif/4318.geoJSON"
user["aoi_shp"]=os.path.splitext(user["aoi_geojson"])[0]+".shp"

In [9]:
import geopandas
gdf=geopandas.read_file(user["aoi_geojson"])
gdf.to_file(user["aoi_shp"])

Here after:
- Enter the path to .shp files where the resulting osm layer will be saved.

In [90]:
## Enter the path to the .shp file where the resulting osm layer will be saved - Path for LINES
user["output_lineshp"]="/Users/taisgrippa/Dropbox/BROL/Mapathon_Tais/2018_gif/Mapathon2018_belgiumuni/output_line.shp"

In [91]:
## Enter the path to the .shp file where the resulting osm layer will be saved - Path for POLYGONS
user["output_polygonshp"]="/Users/taisgrippa/Dropbox/BROL/Mapathon_Tais/2018_gif/Mapathon2018_belgiumuni/output_polygon.shp"

Here after:
- Postgresql tables' name are saved as variables. You could change it only if you want.

In [85]:
## Save postgresqgl table's mane
user["aoi"] = "aoi"
user["bbox"] = "bbox"
user["bbox_coord"] = "bbox_coord"

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Import data in Postgis

## Create postgis extension

Postgis extension will be created if it doesn't exists!

## Import AOI shapefile

The following part use the "shp2pgsql" program which should normally already be installed since postgis extension have been created in postgresql. See [this quick guide](http://www.bostongis.com/pgsql2shp_shp2pgsql_quickguide.bqg) for more information. 

Fist, a bash file is created according to user parameters. Then, the bash.dat file is pass in an embedded command prompt.

In [45]:
from subprocess import Popen, PIPE, STDOUT
p = Popen(['shp2pgsql','-s','3857','-d','-I',user["aoi_shp"],user["schema"]+"."+user["aoi"],
           "|","psql","-d",user["dbname"],'-h',user["host"],'-U',user["user"]], stdout=PIPE, stdin=PIPE, stderr=STDOUT)    
bash_stdout = p.communicate(input=user["password"])[0]
print(bash_stdout.decode())

Shapefile type: Polygon
Postgis type: MULTIPOLYGON[2]
SET CLIENT_ENCODING TO UTF8;
SET STANDARD_CONFORMING_STRINGS TO ON;
SELECT DropGeometryColumn('belgianmapath2018','aoi','geom');
DROP TABLE IF EXISTS "belgianmapath2018"."aoi";
BEGIN;
CREATE TABLE "belgianmapath2018"."aoi" (gid serial,
"fid" float8);
ALTER TABLE "belgianmapath2018"."aoi" ADD PRIMARY KEY (gid);
SELECT AddGeometryColumn('belgianmapath2018','aoi','geom','3857','MULTIPOLYGON',2);
INSERT INTO "belgianmapath2018"."aoi" ("fid",geom) VALUES ('0','...');
CREATE INDEX ON "belgianmapath2018"."aoi" USING GIST ("geom");
COMMIT;
ANALYZE "belgianmapath2018"."aoi";



#### Update the PostGis tables nearly creadted to ensure all geometries are valid

In [48]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

# Query
query="UPDATE "+user["schema"]+"."+user["aoi"]+" \
SET geom = ST_Transform(ST_Multi(ST_CollectionExtract(ST_MakeValid(geom), 3)),3857) \
WHERE ST_IsValid(geom) is not True"
# Execute the CREATE TABLE query 
cur.execute(query)
# Make the changes to the database persistent
db.commit()
    
# Close cursor and communication with the database
cur.close()
db.close()

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

## Create the bounding box (bbox) of the AOI

In [49]:
# Connect to the database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

In [50]:
# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+user["bbox"])
# Make the changes to the database persistent
db.commit()

In [51]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

# Subquery1 
subquery1="SELECT ST_Envelope(geom) as the_geom FROM "+user["schema"]+"."+user["aoi"]

# Save the previous subquery results in a new table
query="CREATE TABLE "+user["schema"]+"."+user["bbox"]+" AS ("
query+=subquery1+")"

# Execute the query 
cur.execute(query)
# Make the changes to the database persistent
db.commit()
# Close cursor and communication with the database
cur.close()
db.close()

## Get upper, lower, right and left coordinates of the bbox (in WGS84).

In [52]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

In [53]:
# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+user["bbox_coord"])
# Make the changes to the database persistent
db.commit()

In [54]:
### Save the previous subquery1 results in a new table
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

subquery1="WITH \
wgs84geom AS(\
SELECT  ST_Transform(the_geom, 4326) AS the_geom \
FROM "+user["schema"]+"."+user["bbox"]+") \
\
SELECT ST_Xmin(the_geom) AS west, ST_Xmax(the_geom) AS east, \
ST_Ymin(the_geom) AS south, ST_Ymax(the_geom) AS north \
FROM wgs84geom"

# Query
query="CREATE TABLE "+user["schema"]+"."+user["bbox_coord"]+" AS ("
query+=subquery1+")"

# Execute the CREATE TABLE query 
cur.execute(query)
# Make the changes to the database persistent
db.commit()
# Close cursor and communication with the database
cur.close()
db.close()

In [55]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Number of line to show (please limit to 100 for saving computing time)
nbrow=500
# Query
query="SELECT * FROM "+user["schema"]+"."+user["bbox_coord"]+" \
LIMIT "+str(nbrow)
# Execute query through panda
df_bbox_coord=pd.read_sql(query, db)
# Show dataframe
df_bbox_coord.head(50)

,west,east,south,north
0,6.17157,6.462708,8.427546,9.704996


In [56]:
## Save coordinates in variables
north_coord=round(df_bbox_coord.loc[0,['north']],6)
south_coord=round(df_bbox_coord.loc[0,['south']],6)
west_coord=round(df_bbox_coord.loc[0,['west']],6)
east_coord=round(df_bbox_coord.loc[0,['east']],6)

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Download OpenStreetMap data using Xapi

Here after, the OpenStreetMap (OSM) data will be downloaded for the AOI. For this prupose, OSM Xapi is used. Please read [this page](http://wiki.openstreetmap.org/wiki/Xapi) or [this one](http://wiki.openstreetmap.org/wiki/Overpass_API/XAPI_Compatibility_Layer) for extra information on Xapi. Please notice that the '@meta' is important to download informations about the osm features (last user, timestamp). Osm2pgsql use a .style file which manage how to osm data will be uploaded in the Postgis database. Please use [this .style file](https://github.com/tgrippa/Mapathon_HOT_OSM_WhatWeMapped/blob/master/default_OSMmetadata.style) wich allow to upload the metadata correctly.

In [56]:
import urllib

## Create the Xapi URL to download the data
print "OSM data will be downloaded for bbox (W:"+str(west_coord)+" S:"+str(south_coord)+" E:"+str(east_coord)+" N:"+str(north_coord)+")"

osm_api_base_url="http://www.overpass-api.de/api/xapi?*"
osm_api_url=osm_api_base_url+"[bbox="+str(west_coord)+","+str(south_coord)+","+str(east_coord)+","+str(north_coord)+"][@meta]"

osm_file=user["osmfolder"]+"/"+user["prefixosm"]+".osm"

## Retrieve the .osm file with the URL
urllib.urlretrieve (osm_api_url, osm_file)
print osm_api_url

OSM data will be downloaded for bbox (W:6.17157 S:8.427546 E:6.462708 N:9.704996)
http://www.overpass-api.de/api/xapi?*[bbox=6.17157,8.427546,6.462708,9.704996][@meta]


# Import OSM layers in PostGis database

In [34]:
## Makes a 5sec stop in the process
import time
time.sleep(5)

## Build the osm2pgsql command-line
cmdline=""
cmdline+="osm2pgsql -c -d "+user["dbname"]+" " 
cmdline+="-W "+" "
cmdline+="-U "+user["user"]+" "
cmdline+="-H "+user["host"]+" "
cmdline+="--extra-attributes"+" "
cmdline+="-p "+user["prefixosm"]+" "
cmdline+="-S "+user["stylefile"]+" "
cmdline+=user["osmfolder"]+"/"+user["prefixosm"]+".osm"

## Create temp bash file for osm2pgsql
outputcsv=os.path.join(user["osmfolder"],"tmp_bash2.sh")
f = open(outputcsv, 'w')
f.write(cmdline)
f.close()

In [42]:
from subprocess import Popen, PIPE, STDOUT
p = Popen(['osm2pgsql','-c','-d',user["dbname"],'-W','-U',user["user"],
           '-H',user["host"],'--extra-attributes','-p',user["prefixosm"],
           '-S',user["stylefile"],os.path.join(user["osmfolder"],user["prefixosm"]+".osm")], 
          stdout=PIPE, stdin=PIPE, stderr=STDOUT)    
bash_stdout = p.communicate(input=user["password"])[0]
print(bash_stdout.decode())

osm2pgsql version 0.94.0 (64 bit id space)

Password:
Using built-in tag processing pipeline
Using projection SRS 3857 (Spherical Mercator)
Setting up table: osm_point
Setting up table: osm_line
Setting up table: osm_polygon
Setting up table: osm_roads
Allocating memory for dense node cache
Allocating dense node cache in one big chunk
Allocating memory for sparse node cache
Sharing dense sparse
Node-cache: cache=800MB, maxblocks=12800*65536, allocation method=3

Reading in file: /Users/taisgrippa/Dropbox/BROL/Mapathon_Tais/2018_gif/Mapathon2018_belgiumuni/osm.osm
Using XML parser.
Processing: Node(596k 33.1k/s) Way(60k 20.00k/s) Relation(0 0.00/s)  parse time: 22s
Node stats: total(596474), max(5518117022) in 18s
Way stats: total(79716), max(574981809) in 3s
Relation stats: total(31), max(8142583) in 0s
Committing transaction for osm_point
Committing transaction for osm_line
Committing transaction for osm_polygon
Committing transaction for osm_roads
Using built-in tag processing pipeli

In [198]:
## Erase the temporary 'reclass_rule.csv' file
os.remove(outputcsv)

Here after, the osm table "line" and "polygon" will be moved to the current user's schema.

As HOT's tasks (Humanitarian OpenStreetMap Team) generally focus on highways, buildings and residential area, these features are the only one kept for the rest of this notebook. If you need something else, you should adapt the code.

In [62]:
## Move tables "lines" and "polygons" in current schema

# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+user["prefixosm"]+"_"+"line")
# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+user["prefixosm"]+"_"+"polygon")
# Make the changes to the database persistent
db.commit()

# Copy table from public schema to current schema (only intersections with AOI):
subquery1="WITH \
lines AS(\
SELECT  (ST_Dump(ST_Intersection(ST_Transform(l.way,4326), ST_Transform(aoi.geom,4326)))).geom AS the_geom, l.*  \
FROM "+user["schema"]+"."+user["aoi"]+" AS aoi, public."+user["prefixosm"]+"_"+"line"+" AS l \
WHERE l.highway is not null) \
\
SELECT * FROM lines"
# Query
query="CREATE TABLE "+user["schema"]+"."+user["prefixosm"]+"_"+"line AS ("
query+=subquery1+")"
# Execute the CREATE TABLE query 
cur.execute(query)

# Copy table from public schema to current schema (only intersections with AOI):
subquery1="WITH \
poly AS(\
SELECT  (ST_Dump(ST_Intersection(ST_Transform(p.way,4326), ST_Transform(aoi.geom,4326)))).geom AS the_geom, p.*  \
FROM "+user["schema"]+"."+user["aoi"]+" AS aoi, public."+user["prefixosm"]+"_"+"polygon"+" AS p \
WHERE  p.building is not NULL OR p.landuse = 'residential' ) \
\
SELECT * FROM poly"
# Query
query="CREATE TABLE "+user["schema"]+"."+user["prefixosm"]+"_"+"polygon AS ("
query+=subquery1+")"
# Execute the CREATE TABLE query 
cur.execute(query)
# Make the changes to the database persistent
db.commit()

# Drop 'way' column if exists:
cur.execute("ALTER TABLE "+user["schema"]+"."+user["prefixosm"]+"_"+"line DROP COLUMN IF EXISTS way")
# Drop table if exists:
cur.execute("ALTER TABLE "+user["schema"]+"."+user["prefixosm"]+"_"+"polygon DROP COLUMN IF EXISTS way")
# Make the changes to the database persistent
db.commit()

Here after, the table in the "public" schema will be droped.

In [63]:
## Drop tables of public schema which will not be used anymore

# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS public."+user["prefixosm"]+"_"+"point")
# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS public."+user["prefixosm"]+"_"+"roads")
# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS public."+user["prefixosm"]+"_"+"line")
# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS public."+user["prefixosm"]+"_"+"polygon")

# Make the changes to the database persistent
db.commit()

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Timestamp for Qgis timemanager plugin

## Add new colum 'forever' for Qgis timemanager

In [64]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

# Query:
cur.execute("ALTER TABLE "+user["schema"]+"."+user["prefixosm"]+"_"+"line ADD COLUMN IF NOT EXISTS forever timestamp(0) with time zone")
cur.execute("ALTER TABLE "+user["schema"]+"."+user["prefixosm"]+"_"+"polygon ADD COLUMN IF NOT EXISTS forever timestamp(0) with time zone")

# Make the changes to the database persistent
db.commit()

In [65]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

# Query:
cur.execute("UPDATE "+user["schema"]+"."+user["prefixosm"]+"_"+"line SET forever=TIMESTAMP WITH TIME ZONE '2030-01-01 00:00:00+01'")
cur.execute("UPDATE "+user["schema"]+"."+user["prefixosm"]+"_"+"polygon SET forever=TIMESTAMP WITH TIME ZONE '2030-01-01 00:00:00+01'")
# Make the changes to the database persistent
db.commit()

# Export shapefile

Here after, the tables 'line' and 'polygon' are exported from Postgis as shapefiles for gentle use in Qgis.

### Export lines table

In [100]:
from subprocess import Popen, PIPE, STDOUT
p = Popen(['pgsql2shp','-f',user["output_lineshp"],
           "-h",user["host"],"-u",user["user"],"-P",user["password"],
           user["dbname"],user["schema"]+"."+user["prefixosm"]+"_"+"line"], 
          stdout=PIPE, stdin=PIPE, stderr=STDOUT)    
bash_stdout = p.communicate(input=user["password"])[0]
print(bash_stdout.decode())

Initializing... 
Done (postgis major version: 2).
Output shape: PolyLine
Dumping: XXXXXXXXXXXXXXXXXXXXX [2056 rows].



### Export polygons table

In [102]:
from subprocess import Popen, PIPE, STDOUT
p = Popen(['pgsql2shp','-f',user["output_polygonshp"],
           "-h",user["host"],"-u",user["user"],"-P",user["password"],
           user["dbname"],user["schema"]+"."+user["prefixosm"]+"_"+"polygon"], 
          stdout=PIPE, stdin=PIPE, stderr=STDOUT)    
bash_stdout = p.communicate(input=user["password"])[0]
print(bash_stdout.decode())

Initializing... 
Done (postgis major version: 2).
Output shape: Polygon
Dumping: XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX [74317 rows].



# Mapathon statistics

Here after, some basic stats are computed to inform on key infos of the mapathon. You could add extra stats if you want.

In [106]:
# Enter starting time of the mapathon (YYYY-MM-DD HH:MM:SS)
begintimemapping='2018-03-24 08:00:00'
# Enter ending time of the mapathon (YYYY-MM-DD HH:MM:SS)
endingtimemapping='2018-03-24 18:00:00'

### Number of unique mapper

In [107]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Number of line to show (please limit to 100 for saving computing time)
nbrow=500
# Query
query="WITH distinctuser_line AS(\
SELECT DISTINCT osm_user FROM "+user["schema"]+"."+user["prefixosm"]+"_"+"line \
WHERE osm_timestamp >= '"+str(begintimemapping)+"' AND osm_timestamp <= '"+str(endingtimemapping)+"'), \
\
distinctuser_polygon AS(\
SELECT DISTINCT osm_user FROM "+user["schema"]+"."+user["prefixosm"]+"_"+"polygon \
WHERE osm_timestamp >= '"+str(begintimemapping)+"' AND osm_timestamp <= '"+str(endingtimemapping)+"'), \
\
global_distinctuser AS(\
SELECT * FROM distinctuser_line \
UNION \
SELECT * FROM distinctuser_polygon)\
\
SELECT count(*) FROM global_distinctuser"

# Execute query through panda
df_bbox_coord=pd.read_sql(query, db)
# Show dataframe
df_bbox_coord.head(50)

,count
0,263


### Number of editions by mapper

In [157]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Number of line to show (please limit to 100 for saving computing time)
nbrow=500
# Query
query="WITH \
all_edits AS(\
SELECT * FROM "+user["schema"]+"."+user["prefixosm"]+"_"+"line \
UNION \
SELECT * FROM "+user["schema"]+"."+user["prefixosm"]+"_"+"polygon) \
\
SELECT osm_user, count(*) AS count FROM all_edits \
WHERE osm_timestamp >= '"+str(begintimemapping)+"' AND osm_timestamp <= '"+str(endingtimemapping)+"' \
GROUP BY osm_user \
ORDER BY count(*) DESC"
# Execute query through panda
df_bbox_coord=pd.read_sql(query, db)
# Show dataframe
df_bbox_coord.head(50)

,osm_user,count
0,Céline94,1195
1,leril,1170
2,ArticCynda,1118
3,fpriem,1047
4,evdegaer,900
5,Klara Dvorakova,862
6,Thomas Vandecappelle,844
7,smossoux2,800
8,Freez09,796
9,WendyR,782


In [158]:
listuser=df_bbox_coord['osm_user']
listcount=df_bbox_coord['count']
mardowntable=[]
mardowntable.append("rank|user|edits")
mardowntable.append("--- | --- | ---")
[mardowntable.append("%s|%s|%s"%(i,a[0],a[1])) for i,a in enumerate(zip(listuser,listcount),1)]
print "\n".join(mardowntable)

rank|user|edits
--- | --- | ---
1|Céline94|1195
2|leril|1170
3|ArticCynda|1118
4|fpriem|1047
5|evdegaer|900
6|Klara Dvorakova|862
7|Thomas Vandecappelle|844
8|smossoux2|800
9|Freez09|796
10|WendyR|782
11|Georgery Florent|742
12|JulietteROME|738
13|Chrilou|718
14|lolo2472|707
15|savleesch|704
16|Vanille07|699
17|Barthélémy Haincourt|688
18|Geogmapper|667
19|jj_geo|666
20|Meryem El|642
21|Notanumber8|642
22|Tim Couwelier|628
23|Guillaume3|618
24|Pauline17|594
25|Steven1555|590
26|Nervang|586
27|JShrek|577
28|HildeVerb|566
29|François Rigo|562
30|Lucas Wpch|561
31|Jules19|560
32|Ubipo|548
33|Emy170|545
34|RobertBau|537
35|Massimo Camastra|535
36|AlizéeS|534
37|manon muratore|531
38|Rahimi Samim|521
39|Rulus|518
40|JustineEloy|516
41|pauline500|491
42|Anne DW|484
43|aline legrand|477
44|MrFrazil|476
45|broussieau|465
46|cstal|463
47|jobzzi|463
48|eli2s|455
49|ccloquet|446
50|Florence L|445
51|TristanSatinet|437
52|Daamiien|430
53|GertVanhollebeke|430
54|Mathieud3|426
55|Slvn01|419
56|Justi

### Number of buildings mapped

In [109]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Number of line to show (please limit to 100 for saving computing time)
nbrow=500
# Query
query="SELECT count(*) FROM "+user["schema"]+"."+user["prefixosm"]+"_"+"polygon \
WHERE osm_timestamp >= '"+str(begintimemapping)+"' AND osm_timestamp <= '"+str(endingtimemapping)+"' \
AND building is not null"
# Execute query through panda
df_bbox_coord=pd.read_sql(query, db)
# Show dataframe
df_bbox_coord.head(50)

,count
0,70970


### Number of Buildings by mapper

In [161]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Number of line to show (please limit to 100 for saving computing time)
nbrow=500
# Query
query="\
SELECT osm_user, count(*) AS count FROM "+user["schema"]+"."+user["prefixosm"]+"_"+"polygon \
WHERE osm_timestamp >= '"+str(begintimemapping)+"' AND osm_timestamp <= '"+str(endingtimemapping)+"' \
AND building IS NOT NULL \
GROUP BY osm_user \
ORDER BY count(*) DESC"
# Execute query through panda
df_bbox_coord=pd.read_sql(query, db)
# Show dataframe
df_bbox_coord.head(50)

,osm_user,count
0,Céline94,1190
1,leril,1148
2,ArticCynda,1048
3,fpriem,1033
4,evdegaer,899
5,Klara Dvorakova,853
6,Thomas Vandecappelle,838
7,smossoux2,792
8,Freez09,772
9,WendyR,767


In [163]:
listuser=df_bbox_coord['osm_user']
listcount=df_bbox_coord['count']
mardowntable=[]
mardowntable.append("rank|user|buildings")
mardowntable.append("--- | --- | ---")
[mardowntable.append("%s|%s|%s"%(i,a[0],a[1])) for i,a in enumerate(zip(listuser,listcount),1)]
print "\n".join(mardowntable)

rank|user|buildings
--- | --- | ---
1|Céline94|1190
2|leril|1148
3|ArticCynda|1048
4|fpriem|1033
5|evdegaer|899
6|Klara Dvorakova|853
7|Thomas Vandecappelle|838
8|smossoux2|792
9|Freez09|772
10|WendyR|767
11|Georgery Florent|730
12|JulietteROME|717
13|Chrilou|710
14|savleesch|704
15|lolo2472|684
16|Barthélémy Haincourt|680
17|Vanille07|678
18|Geogmapper|663
19|Meryem El|642
20|Notanumber8|633
21|Tim Couwelier|628
22|jj_geo|622
23|Guillaume3|617
24|Pauline17|577
25|JShrek|575
26|Steven1555|573
27|HildeVerb|563
28|Emy170|534
29|Nervang|533
30|Jules19|532
31|AlizéeS|525
32|manon muratore|522
33|Rahimi Samim|517
34|Rulus|515
35|Massimo Camastra|508
36|RobertBau|503
37|François Rigo|501
38|Lucas Wpch|489
39|JustineEloy|488
40|Anne DW|479
41|pauline500|476
42|aline legrand|475
43|MrFrazil|468
44|broussieau|465
45|Ubipo|462
46|eli2s|445
47|ccloquet|441
48|TristanSatinet|429
49|jobzzi|428
50|GertVanhollebeke|427
51|Florence L|427
52|Mathieud3|419
53|Slvn01|417
54|Justine Cloquette|417
55|Clem2

### Number of residential area mapped

In [110]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Number of line to show (please limit to 100 for saving computing time)
nbrow=500
# Query
query="SELECT count(*) FROM "+user["schema"]+"."+user["prefixosm"]+"_"+"polygon \
WHERE osm_timestamp >= '"+str(begintimemapping)+"' AND osm_timestamp <= '"+str(endingtimemapping)+"' \
AND landuse='residential'"
# Execute query through panda
df_bbox_coord=pd.read_sql(query, db)
# Show dataframe
df_bbox_coord.head(50)

,count
0,1263


### Cumulated area of residential area mapped

In [120]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Number of line to show (please limit to 100 for saving computing time)
nbrow=500
# Query
query="SELECT sum(ST_Area(ST_Transform(the_geom,"+ESPG_UTM+")))/1000000 AS km2_residential FROM "+user["schema"]+"."+user["prefixosm"]+"_"+"polygon \
WHERE osm_timestamp >= '"+str(begintimemapping)+"' AND osm_timestamp <= '"+str(endingtimemapping)+"' \
AND landuse='residential'"
# Execute query through panda
df_bbox_coord=pd.read_sql(query, db)
# Show dataframe
df_bbox_coord.head(50)

,km2_residential
0,29.744125


### Number of roads mapped

In [111]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Number of line to show (please limit to 100 for saving computing time)
nbrow=500
# Query
query="SELECT count(*) FROM "+user["schema"]+"."+user["prefixosm"]+"_"+"line \
WHERE osm_timestamp >= '"+str(begintimemapping)+"' AND osm_timestamp <= '"+str(endingtimemapping)+"' \
AND highway is not null"
# Execute query through panda
df_bbox_coord=pd.read_sql(query, db)
# Show dataframe
df_bbox_coord.head(50)

,count
0,1148


### Cumulated distance of roads mapped

In [116]:
ESPG_UTM="32632"

In [118]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Number of line to show (please limit to 100 for saving computing time)
nbrow=500
# Query
query="SELECT sum(ST_Length(ST_Transform(the_geom,"+ESPG_UTM+")))/1000 AS km_road FROM "+user["schema"]+"."+user["prefixosm"]+"_"+"line \
WHERE osm_timestamp >= '"+str(begintimemapping)+"' AND osm_timestamp <= '"+str(endingtimemapping)+"' \
AND highway is not null"
# Execute query through panda
df_bbox_coord=pd.read_sql(query, db)
# Show dataframe
df_bbox_coord.head(50)

,km_road
0,277.371382


### Cumulated distance of road by mapper

In [164]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Number of line to show (please limit to 100 for saving computing time)
nbrow=500
# Query
query="\
SELECT osm_user, sum(ST_Length(ST_Transform(the_geom,"+ESPG_UTM+")))/1000 AS km_road FROM "+user["schema"]+"."+user["prefixosm"]+"_"+"line \
WHERE osm_timestamp >= '"+str(begintimemapping)+"' AND osm_timestamp <= '"+str(endingtimemapping)+"' \
AND highway IS NOT NULL \
GROUP BY osm_user \
ORDER BY km_road DESC"
# Execute query through panda
df_bbox_coord=pd.read_sql(query, db)
# Show dataframe
df_bbox_coord.head(50)

,osm_user,km_road
0,Pieter Vander Vennet,26.780207
1,Bosco_Bwambale,17.843073
2,Ubipo,16.499218
3,Patcach-be,14.711452
4,phy,13.529825
5,cstal,13.249746
6,SLadouce,11.822905
7,AmélieP,10.593650
8,lolo2472,10.013440
9,Lucas Wpch,8.765242


In [166]:
listuser=df_bbox_coord['osm_user']
listcount=df_bbox_coord['km_road']
mardowntable=[]
mardowntable.append("rank|user|km_road")
mardowntable.append("--- | --- | ---")
[mardowntable.append("%s|%s|%s"%(i,a[0],round(a[1],2))) for i,a in enumerate(zip(listuser,listcount),1)]
print "\n".join(mardowntable)

rank|user|km_road
--- | --- | ---
1|Pieter Vander Vennet|26.78
2|Bosco_Bwambale|17.84
3|Ubipo|16.5
4|Patcach-be|14.71
5|phy|13.53
6|cstal|13.25
7|SLadouce|11.82
8|AmélieP|10.59
9|lolo2472|10.01
10|Lucas Wpch|8.77
11|François Rigo|7.86
12|ArticCynda|7.49
13|jj_geo|7.21
14|Jorieke V|6.17
15|RobertBau|6.05
16|Nervang|5.79
17|jobzzi|4.88
18|TG22|4.85
19|Anouk Cauwels|4.74
20|Timon Deputter|4.08
21|ChloéYamina|3.91
22|Elwymo|3.81
23|michiel maertens|3.63
24|khadmi|3.48
25|Stani2018|3.26
26|JaninaClark|2.38
27|AlizéeS|2.31
28|Emilie w|2.28
29|Madeleine Guyot|2.16
30|Geogmapper|2.07
31|Emy170|1.98
32|valthebestIII|1.95
33|DVercamer|1.78
34|HOUNKPE Joseph|1.71
35|Eleuthere|1.68
36|Massimo Camastra|1.59
37|Florence L|1.55
38|Eléonore Wolff|1.5
39|Georgery Florent|1.44
40|joost schouppe|1.41
41|Francoise Skirole|1.18
42|jbelien|1.18
43|Zappi211|1.17
44|laurentthomas|1.16
45|JPL|1.15
46|Georges Duculot|1.14
47|Vanille07|1.14
48|TimTherry|1.09
49|Loïs Magain|1.05
50|Sara Brancart|0.94
51|De Berral

# Create map frames using Time Manager Plugin

**This step should be done outsite from this notebook.**

Use the [Qgis time manager plugin](https://github.com/anitagraser/TimeManager) to produced maps frames. More info could be found on the [Anita Graser's blog](https://anitagraser.com/2011/11/20/nice-animations-with-time-managers-offset-feature/). 

In the following cell of this notebook, please enter the path to the folder containing the frame for .gif and .mp4 creation purpose. More infos for this step can be found [here](http://stackoverflow.com/questions/753190/programmatically-generate-video-or-animated-gif-in-python).

In [170]:
## Enter the path to the folder with maps frames from TimeManager Plug-in
user["map_frames_folder"]='/Users/taisgrippa/Dropbox/BROL/Mapathon_Tais/2018_gif/Time_manager/frames'

## Create GIF

For creation of .gif output, please first install [imageio](https://github.com/imageio/imageio).

In [175]:
import imageio
import glob
frames = []
framepattern=user["map_frames_folder"]+"/*.png"
frames=glob.glob(framepattern)

In [176]:
from PIL import Image
from numpy import array
imagestack = []
for frame in frames:
    img = Image.open(frame)
    arr = array(img)
    imagestack.append(arr)
imageio.mimsave(user["map_frames_folder"]+'/Mapathon2016accomplishment.gif', imagestack, fps=4)